## 2.Importación de librerías

In [21]:
# la librería Tweepy permitirá acceder a Twitter via las credenciales de developer
from tweepy import OAuthHandler #logearse
from tweepy.streaming import StreamListener
import tweepy, json
# Importamos nuestras claves de acceso a Twitter desde el archivo credenciales
# Tenerlas en un archivo nos permite manejarlo como variables

from credentials import * 
import pandas as pd 
import numpy as np

## 3. Configuración de accesso Twitter API

In [25]:
# configuracion del API
def twitter_setup():
    """
    Esta funcion permite configurar el API de Twitter
    con nuestras credenciales como developer
    """

    # Autenticacion y acceso utilizando las claves
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN ,ACCESS_SECRET)

    # Developer API con autentication
    api = tweepy.API(auth)
    return api

## 4. Extraccion de Tweets

**OPCION 1:** Usamos la función **extractor** de Tweepy.

Extraemos los tweets de un usuario específico. Se especifica en **screen_name** el nombre del usuario, **count** la cantidad de tweets.

In [26]:
# Extracción de tweets
# Creamos el objeto_extractor
extractor = twitter_setup()

# Creamos una lista de tweets de la siguiente manera
tweets = extractor.user_timeline(screen_name="Coca Cola", count=50)
print("Número de tweets extraidos: {}.\n".format(len(tweets)))

#Visualisamo los tweets los 5 tweets mas recientes
print('5 tweets recientes:', end="\n\n")
print(tweets)
for tweet in tweets[:5]:
    print(tweet.text, end="\n\n")

TweepError: [{'code': 89, 'message': 'Invalid or expired token.'}]

In [13]:
# correr el código otro notebook
%run ./credentials.ipynb

**OPCION 2:** Tambien se puede usar **MyStreamListener** de tweepy

Extraemos los tweets de acuerdo a las palabras clave que se especifiquen (independientemente del nombre del usuario)

Despues de obtener accesso a los datos de Twitter, ahora crearemos un archivo para guardar todos los tweets encontrados.

In [24]:
 # Autenticacion y acceso utilizando las claves:
auth_l = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth_l.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

In [19]:
tweet_list=[]
class MyStreamListener(tweepy.StreamListener):
    def __init__(self,api=None):
        super(MyStreamListener,self).__init__()
        self.num_tweets=0
        self.file=open("tweet.txt","w")
        
    def on_status(self,status):
        tweet=status._json
        self.file.write(json.dumps(tweet)+ '\n')
        tweet_list.append(status)
        self.num_tweets+=1
        if self.num_tweets<1000:
            return True
        else:
            return False
        self.file.close()

In [20]:
# Creamos un objeto de transmisión y autenticamos
l = MyStreamListener() 
stream = tweepy.Stream(auth_l, l)
# esta línea filtra flujos de datos en Twitter para capturar datos por palabras clave
stream.filter (track = ['covid', 'corona', 'covid19', 'coronavirus', 
'mascarilla', 'desinfectante', 'distanciamiento social'])

stream.filter (track = ['fraudeenmesa','keiko fujimori','pedro castillo',
                        'conteo','actas','sagasti'])

In [27]:
tweets_data_path='tweet copy.txt'
tweets_data=[]
tweets_file=open(tweets_data_path,"r")
# Leemos los tweets y los pasamos a una lista
for line in tweets_file:
    tweet=json.loads(line)
    tweets_data.append(tweet)
tweets_file.close()
# Visualizamos el contenido del primer tweet
print(tweets_data[0])

{'created_at': 'Mon Jun 21 05:24:02 +0000 2021', 'id': 1406845342650277890, 'id_str': '1406845342650277890', 'text': 'RT @ParisHilton: 🎥 I’m so honored to be showing my #ThisIsParis documentary directed by Alexandra Dean on closing night of @Tribeca Film Fe…', 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 980630189406224384, 'id_str': '980630189406224384', 'name': 'Grandma Crow', 'screen_name': 'sovereignsouls', 'location': 'United States', 'url': None, 'description': '🌎 Internet Bill of Rights 🌎\nhttps://www.eff.org/cyberspace-independence', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 2342, 'friends_count': 2958, 'listed_count': 2, 'favourites_count': 66816, 'statuses_count': 42645, 'created_at': 'Mon Apr 02 02:17

In [28]:
# Extraemos algunas variables relevantes
ids = [tweet['id_str'] for tweet in tweets_data]
times = [tweet['created_at'] for tweet in tweets_data]
users = [tweet['user']['name'] for tweet in tweets_data]
texts = [tweet['text'] for tweet in tweets_data]
lats = [(T['geo']['coordinates'][0] if T['geo'] else None) for T in tweets_data]
lons = [(T['geo']['coordinates'][1] if T['geo'] else None) for T in tweets_data]
place_names = [(T['place']['full_name'] if T['place'] else None) for T in tweets_data]
place_types = [(T['place']['place_type'] if T['place'] else None) for T in tweets_data]

In [29]:
ids[0],times[0], users[0], texts[0]

('1406845342650277890',
 'Mon Jun 21 05:24:02 +0000 2021',
 'Grandma Crow',
 'RT @ParisHilton: 🎥 I’m so honored to be showing my #ThisIsParis documentary directed by Alexandra Dean on closing night of @Tribeca Film Fe…')

In [30]:
import csv #import csv 
import os 

In [31]:
api = twitter_setup()

csvFile = open("segunda_vuelta1.csv", 'a', newline='', encoding='utf-8')
try:
    writer = csv.writer(csvFile, quoting=csv.QUOTE_ALL)
    writer.writerow(["created_at", "text", "len", "likes", "RTs", "user_name"])
    
    for tweet in tweepy.Cursor(api.search,
                           q = "#fraudeenmesa -filter:retweets", count=5000,
                           since = "2021-06-8",
                           until = "2021-06-14",
                           lang = "es").items():
        writer.writerow([tweet.created_at, tweet.text,
                        tweet.favorite_count, tweet.retweet_count, tweet.user.screen_name])
finally:
    csvFile.close()


TweepError: Twitter error response: status code = 401

In [32]:
# Creamos el dataframe de pandas desde la lista de tweets obtenidos mediante las credenciales de Twitter 
# y contenidos en la lista tweets:
data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

# Mostramos las 10 primeras observaciones del dataframe:
display(data.head(10))

,Tweets
